In [1]:
import pandas as pd
import numpy as np
import math
import datetime
import time
import warnings
from ipynb.fs.defs.functions import new_route
from ipynb.fs.defs.functions import get_row
from ipynb.fs.defs.functions import get_column
from ipynb.fs.defs.functions import getName
from ipynb.fs.defs.functions import timeDistance
from ipynb.fs.defs.functions import best_index
from ipynb.fs.defs.functions import find_best_position
from ipynb.fs.defs.functions import cities_list
from ipynb.fs.defs.functions import random_schedule_day
warnings.filterwarnings('ignore')

table=pd.read_csv("../Dati/orari/route.csv", delimiter=";")
time_table=pd.read_csv("../Dati/orari/route_time.csv", delimiter=";")
dft = pd.read_csv("../Dati/distanze/corrected_times_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
dfd = pd.read_csv("../Dati/distanze/distances_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)

In [2]:
#(inizio e fine compresi)
#Ritorna una lista contenente le chiavi di due città e il tempo di percorrenza tra le due
#nel formato (times, "key1 - key2")
#"key1 - key2" è la chiave
#La lista partirà dalla posizione "inizio" della schedulazione e finità con "fine"
def route(schedule, inizio, fine, dft):
    times = []
    keys = []
    row = dft[get_row(dft, schedule[inizio]).values]
    key1 = ''
    key2 = ''
    impossibile = []
    
    for i in range(inizio+1,fine+1):
        key1 = row.index[0]
        try:
            paese=schedule[i]
            if 'Martignana' in paese:
                paese = 'Martignana di Po'
        except:
            continue
        #Distanze in tempo dal paese corrente di ognuno dei furgoni alla destinazione
        value=row.loc[:,get_column(dft, paese).values].values[0][0]
        times.append(round(value))
        row = dft[get_row(dft, paese).values]
        key2 = row.index[0]
        keys.append(key1 + ' $ ' + key2)
    
    steps_times1 = pd.Series(times , keys)
    return steps_times1

In [3]:
#Datogli una schedulazione calcola il tempo di percorrenza
def schedule_time(schedule, dft):
    times = []
    row = dft[get_row(dft, 'Cremona, sesto 39').values]

    for i in range(1, len(schedule)):
        paese=schedule[i]
        if 'Martignana' in paese:
            paese = 'Martignana di Po'

        value=row.loc[:,get_column(dft, paese).values].values[0][0]
        times.append(round(value))
        
        row = dft[get_row(dft, paese).values]
    return sum(times)

In [4]:
#mantiene il nodo inizial e il successivo a y invariati, mentre fa il reverse della lista tra i due
#A-b-c-d-E -> A-d-c-b-E
def cross(schedule, x, y):
    i = [schedule[x]]
    j1 = [schedule[y+1]]
    j1_i1 = list(schedule[x+1: y+1].sort_index(ascending = False).values) 
    return i + j1_i1 + j1

In [5]:
#Data una schedulazione e due nodi di cui fare l'incrocio dei nodi
#effettua la tabulazione del nuovo percorso, dopodichè restituisce il migliore
#tra il nuovo e il vecchio
def two_opt(schedule , i, j, dft):
    #computo la prima parte che rimane uguale
    first_part = route(schedule, 0, i, dft)

    #stesso per l'ultima
    final_part = route(schedule, j+1, len(schedule), dft)
    #print(final_part)
    crossed = cross(schedule, i, j)
    central_part = route(crossed, 0,len(crossed), dft)
    
    new_schedule = first_part.append(central_part).append(final_part)
    if(schedule_time(schedule, dft)> sum(new_schedule.values)):
        return new_schedule
    else:
        return route(schedule, 0, len(schedule), dft)

In [6]:
#Datogli una schedulazione calcola il tempo di percorrenza
def schedule_time(schedule, dft):
    times = []
    row = dft[get_row(dft, schedule[0]).values]

    for i in range(1, len(schedule)):
        paese=schedule[i]
        if 'Martignana' in paese:
            paese = 'Martignana di Po'

        value=row.loc[:,get_column(dft, paese).values].values[0][0]
        times.append(round(value))
        
        row = dft[get_row(dft, paese).values]
    return sum(times)

In [12]:
idx = dft.index[0:35]
dft = pd.read_csv("../Dati/distanze/synthetic_costs.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
dft.columns = idx[:]
dft.index = idx[:]
tries = range(5, 36, 5)
# benchmark_time_s = pd.read_csv("../Dati/altro/synt_benchmark.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
# benchmark_time_s = pd.concat([benchmark_time_s, pd.Series(index=["2opt"], dtype=float)])[[str(x) for x in tries]]
benchmark_counter_s = pd.read_csv("../Dati/altro/synt_benchmark_counter.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
benchmark_counter_s = pd.concat([benchmark_counter_s, pd.Series(index=["2opt2"], dtype=float)])[[str(x) for x in tries]]

for n in tries:
    print(n)
    tot = 0
    counter=0
    schedule = random_schedule_day(dft, n)

    schedule.insert(0, "Agnadello, Marconi, 6")
    schedule.append("Agnadello, Marconi, 6")

    best_schedule = pd.Series(schedule)
    best_time = schedule_time(schedule, dft)
    local_best_time = best_time
    
    while(True):
        startime = time.time()
        for i in range(1, len(schedule)-2):
            for j in range(i+1, len(schedule)-1):
                s = two_opt(pd.Series(schedule), i, j, dft)
                t = sum(s.values)
                if schedule_time(best_schedule, dft)> t:
                    best_time = t.copy()
                    best_schedule = cities_list(s) 
        counter += 1
        tot += (time.time() - startime)
        if(local_best_time>best_time):
            schedule = best_schedule[:]
            local_best_time = best_time
        else:
            break
        
    benchmark_counter_s[str(n)]["2opt2"] = counter
#     benchmark_time_s[str(n)]["2opt"] = round(tot/counter, 3)
    print(counter)
    

5
2
10
6
15
12
20
14
25
18
30
22
35
28


In [66]:
benchmark_counter_s

,5,10,15,20,25,30,35
LS,2.667,7.667,12.0,15.667,18.333,23.333,27.333
2opt,3.000,6.000,10.0,17.000,19.000,29.000,33.000
2opt,NaN,NaN,NaN,NaN,NaN,NaN,33.000


In [7]:
pd.read_csv("../Dati/distanze/synthetic_costs.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,0.00,44.05,75.77,43.46,43.42,79.92,38.60,47.68,86.82,70.94,...,8.06,32.70,34.01,48.92,73.11,84.01,15.03,68.88,58.86,49.25
1,44.05,0.00,36.01,45.40,8.06,77.82,39.12,38.28,78.24,26.91,...,52.01,61.68,14.32,65.03,78.16,51.24,56.94,35.11,30.00,27.46
2,75.77,36.01,0.00,54.92,32.98,71.03,51.86,43.57,65.62,23.35,...,83.77,82.04,41.79,76.16,77.28,20.62,90.14,13.34,22.47,31.89
3,43.46,45.40,54.92,0.00,38.05,37.54,6.71,11.40,43.38,64.54,...,48.76,29.97,32.28,21.54,34.00,52.43,57.28,42.64,32.45,23.77
4,43.42,8.06,32.98,38.05,0.00,69.78,32.02,30.36,70.21,29.68,...,51.48,56.80,9.90,58.31,70.34,45.71,57.38,29.43,22.83,19.42
5,79.92,77.82,71.03,37.54,69.78,0.00,43.84,39.96,12.53,89.11,...,84.08,56.29,67.12,36.35,13.00,57.01,92.57,58.01,52.95,50.70
6,38.60,39.12,51.86,6.71,32.02,43.84,0.00,10.44,48.88,59.48,...,44.55,30.61,25.71,26.48,40.71,51.88,52.95,40.36,29.70,20.00
7,47.68,38.28,43.57,11.40,30.36,39.96,10.44,0.00,42.52,54.45,...,54.15,40.50,27.20,32.65,40.02,41.68,62.39,31.24,21.10,13.15
8,86.82,78.24,65.62,43.38,70.21,12.53,48.88,42.52,0.00,85.73,...,91.76,66.27,69.31,47.01,25.50,49.16,100.30,53.37,50.57,50.80
9,70.94,26.91,23.35,64.54,29.68,89.11,59.48,54.45,85.73,0.00,...,78.87,86.35,39.41,85.87,93.02,43.93,83.49,32.70,36.22,41.30


In [9]:
benchmark_time_s

,5,10,15,20,25,30,35
LS,0.122,0.781,1.808,3.635,5.422,8.785,10.957
greedy,0.028,0.122,0.244,0.412,0.776,1.041,1.684
2opt,0.624,4.639,15.643,37.000,67.034,99.655,NaN


In [11]:
benchmark_counter_s.to_csv("../Dati/altro/synt_benchmark_counter.csv", sep=";", encoding = "ISO-8859-1")
benchmark_time_s.to_csv("../Dati/altro/synt_benchmark.csv", sep=";", encoding = "ISO-8859-1")

In [67]:
import matplotlib.pyplot as plt
plt.figure(figsize=[7,7])
plt.plot(benchmark.loc["2opt"], "bo-")
plt.show()
benchmark.to_csv("../Dati/altro/benchmark.csv", sep=";", encoding = "ISO-8859-1")

NameError: name 'benchmark' is not defined

<Figure size 504x504 with 0 Axes>

In [6]:
pd.read_csv("../Dati/turni_f/fine_optimized_start/foss_ls1.csv", delimiter=";", encoding = "ISO-8859-1")

,lunedi,martedi,mercoledi,giovedi,venerdi
0,"Cremona, Sesto 39","Cremona, Sesto 39","Cremona, Sesto 39","Cremona, Sesto 39","Cremona, Sesto 39"
1,"Izano, Gerardo da Izano, 4/C","Grumello Cremonese ed Uniti, Roma, 2","Sesto ed Uniti, Roma, 4","Annicco, Giuseppe Garibaldi, 6","Cremona, Sesto 39"
2,"Crema, Civerchi, 9","Pizzighettone, Giuseppe Garibaldi, 18","Pizzighettone, Giuseppe Garibaldi, 18","Paderno Ponchielli, Municipio, 3","Cremona, Ugolani Dati, 4"
3,"Capergnanica, XI febbraio, 7","Ostiano, Mazzini, 1","Casalpusterlengo, Pusterla 7","Soresina, Giacomo Matteotti, 6","Gadesco-Pieve Delmona, Giovanni Lonati 9"
4,"Bagnolo Cremasco, Roma, 3",NaN,"Montodine, XXV Aprile, 10","San Bassano, Mons. Frosi","Grontardo, Roma, 16"
5,"Vaiano Cremasco, Gloriosi Caduti, 3",NaN,"Ripalta Cremasca, Roma, 5","Castelleone, Roma, 67","Isola Dovarese, Giacomo Matteotti, 1"
6,"Dovera, Oldrini 2",NaN,"Crema, Civerchi, 9","Fiesco, Giuseppe Parini, 10","Piadena, Platina, 40"
7,"Spino d'Adda, Circonvallazione, 13",NaN,"Pianengo, Nenni","Romanengo, Guaiarini, 14","Solarolo Rainerio, Giuseppina, 81"
8,"Agnadello, Marconi, 6",NaN,"Ricengo, Guglielmo Marconi","Soncino, Dante Alighieri, 23","Cingia de' Botti, Giuseppina, 79"
9,"Trescore Cremasco, Vittorio Veneto, 1",NaN,"Offanengo, Clavelli Martini, 5","Camisano, Trieste, 57","Casalmaggiore, Guglielmo Marconi, 8"


In [6]:
#TIME 2-OPT
week= ['lunedi', 'martedi', 'mercoledi', 'giovedi', 'venerdi']
#results = pd.read_csv("../Dati/altro/computational_times_FOSS.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
#results = pd.concat([results, pd.Series(index=["2opt"])])
fur1 = pd.read_csv("../Dati/turni_f/random_schedule/random_ls_f1.csv", delimiter=";", encoding = "ISO-8859-1")
df = pd.DataFrame({})
for day in week:
    print(day)
    #Otteniamo la schedulazione unendo Cremona-tabella del giorno-Ostiano
    #schedule = pd.Series('Cremona, sesto 39')
    schedule = fur1[day].dropna()
    #schedule = schedule.append(pd.Series('Ostiano, Mazzini, 1' , [len(schedule)]))
    #print(schedule)
    #Otteniamo la schedulazione filtrando i nan e salviamo anche il suo tempo di percorrenza
    best_schedule = pd.Series(schedule.dropna())
    best_time = schedule_time(schedule, dft)
    local_best_time = best_time
    #Per ogni i e j effettua il 2-opt e salva se il risultato migliora con lo scambio
    #Alla fine, avendo provato a scambiare tutte le coppie e avendole confrontate otteremo la miglior schedulazione
    #per ogni giro di 2-opt parte dalla schedulazione greedy
    count = 0
    tot = 0
    while(True):
        #startime = time.time()
        for i in range(1, len(schedule)-2):
            for j in range(i+1, len(schedule)-1):
                s = two_opt(pd.Series(schedule), i, j, dft)
                t = sum(s.values)
                if schedule_time(best_schedule, dft)> t:
                    best_time = t.copy()
                    best_schedule = cities_list(s) 
        #tot += (time.time() - startime)
        #count += 1
        if(local_best_time>best_time):
            schedule = best_schedule[:]
            local_best_time = best_time
        else:
            break
    df = pd.concat([df, pd.DataFrame(pd.Series(best_schedule).rename(day))], axis=1)
    #results[day]["2opt"]=(round((tot/count),3), len(fur1[day].dropna())-2, count)
df.to_csv('../Dati/turni_f/random_schedule/random_2opt_f1.csv', index=False, sep=";", encoding = "ISO-8859-1")
df

lunedi
martedi
mercoledi
giovedi
venerdi


,lunedi,martedi,mercoledi,giovedi,venerdi
0,"Cremona, Sesto 39","Cremona, Sesto 39","Cremona, Sesto 39","Cremona, Sesto 39","Cremona, Sesto 39"
1,"Castelverde, Ubaldo Ferrari, 15","Paderno Ponchielli, Municipio, 3","San Bassano, Mons. Frosi","Cremona, Sesto 39","Persico Dosimo, Albert ,7"
2,"Cremona, Ugolani Dati, 4","Soresina, Giacomo Matteotti, 6","Castelleone, Roma, 67","Sesto ed Uniti, Roma, 4","Gadesco-Pieve Delmona, Giovanni Lonati 9"
3,"Casalpusterlengo, Pusterla 7","Izano, Gerardo da Izano, 4/C","Crema, Civerchi, 9","Grumello Cremonese ed Uniti, Roma, 2","Rivolta d'Adda, Stefano Renzi, 5"
4,"Bagnolo Cremasco, Roma, 3","Crema, Civerchi, 9","Trescore Cremasco, Vittorio Veneto, 1","Annicco, Giuseppe Garibaldi, 6","Ricengo, Guglielmo Marconi"
5,"Ripalta Cremasca, Roma, 5","Vaiano Cremasco, Gloriosi Caduti, 3","Palazzo Pignano, Marco Polo, 31","Casalpusterlengo, Pusterla 7","Madignano, Portici, 4"
6,"Soresina, Giacomo Matteotti, 6","Monte Cremasco, Roma, 12","Pandino, Castello, 15","Montodine, XXV Aprile, 10","Castelleone, Roma, 67"
7,"Soncino, Dante Alighieri, 23","Dovera, Oldrini 2","Agnadello, Marconi, 6","Capergnanica, XI febbraio, 7","Pizzighettone, Giuseppe Garibaldi, 18"
8,"Manerbio, INSURREZIONE NAZIONALE","Pandino, Castello, 15","Capralba, Piave, 2","Spino d'Adda, Circonvallazione, 13","Sospiro, Tosi"
9,"Gadesco-Pieve Delmona, Giovanni Lonati 9","Rivolta d'Adda, Stefano Renzi, 5","Pianengo, Nenni","Crema, Civerchi, 9","Solarolo Rainerio, Giuseppina, 81"


In [10]:
results = results[fur1.columns]
results.to_csv("../Dati/altro/computational_times_FOSS.csv", sep=";", encoding = "ISO-8859-1")
results

,lunedi,martedi,mercoledi,giovedi,venerdi
greedy,"(0.069, 10)","(0.016, 3)","(0.312, 16)","(0.249, 14)","(0.165, 16)"
LS,"(0.0, 10, 1)","(0.0, 2, 1)","(0.0, 17, 1)","(1.496, 16, 2)","(1.546, 16, 2)"
2opt,"(4.692, 10, 2)","(0.03, 2, 1)","(23.985, 17, 6)","(25.002, 16, 1)","(20.613, 16, 8)"


In [7]:
week= ['martedi', 'mercoledi', 'giovedi']
fur2 = pd.read_csv("../Dati/turni_f/random_schedule/random_ls_f2.csv", delimiter=";", encoding = "ISO-8859-1")
df = pd.DataFrame({})
for day in week:
    print(day)
    #schedule = pd.Series('Cremona, sesto 39')
    schedule = fur2[day].dropna()
    #schedule = schedule.append(pd.Series('San Giovanni in Croce, Feudatari, 25 A' , [len(schedule)]))
    #print(schedule)
    best_schedule = pd.Series(schedule.dropna())
    best_time = schedule_time(schedule, dft)
    local_best_time = best_time
    while(True):
        for i in range(1, len(schedule)-2):
            for j in range(i+1, len(schedule)-1):
                s = two_opt(pd.Series(schedule), i, j, dft)
                t = sum(s.values)
                if schedule_time(best_schedule, dft)> t:
                    best_time = t.copy()
                    best_schedule = cities_list(s) 
        if(local_best_time>best_time):
            schedule = best_schedule[:]
            local_best_time = best_time
        else:
            break 
    df = pd.concat([df, pd.DataFrame(pd.Series(best_schedule).rename(day))], axis=1)        
df.to_csv('../Dati/turni_f/random_schedule/random_2opt_f2.csv', index=False, sep=";", encoding = "ISO-8859-1")
df

martedi
mercoledi
giovedi


,martedi,mercoledi,giovedi
0,"Cremona, Sesto 39","Cremona, Sesto 39","Cremona, Sesto 39"
1,"Pizzighettone, Giuseppe Garibaldi, 18","Persico Dosimo, Albert ,7","Cremona, Ugolani Dati, 4"
2,"Cremona, Ugolani Dati, 4","Spino d'Adda, Circonvallazione, 13","Cremona, Palosca n. 2"
3,"Cingia de' Botti, Giuseppina, 79","Crema, Civerchi, 9","Bonemerse, Roma, 25"
4,"Solarolo Rainerio, Giuseppina, 81","Vescovato, Roma, 44","Vescovato, Roma, 44"
5,"Casalmaggiore, Guglielmo Marconi, 8","Pescarolo ed Uniti, Giuseppe Mazzini, 73","Ostiano, Mazzini, 1"
6,"Martignana di Po, Libertà, 64","Ostiano, Mazzini, 1","Piadena, Platina, 40"
7,"Piadena, Platina, 40","San Giovanni in Croce, Feudatari, 25 A","Rivarolo del Re ed Uniti, Giovanni XXIII, 5"
8,"San Giovanni in Croce, Feudatari, 25 A",NaN,"Scandolara Ravara, Italia"
9,"San Giovanni in Croce, Feudatari, 25 A",NaN,"San Giovanni in Croce, Feudatari, 25 A"


In [11]:
#KM 2-OPT
week= ['lunedi', 'martedi', 'mercoledi', 'giovedi', 'venerdi']
df = pd.DataFrame({})
for day in week:
    print(day)
    #Otteniamo la schedulazione unendo Cremona-tabella del giorno-Ostiano
    #schedule = pd.Series('Cremona, sesto 39')
    schedule = fur1d[day].dropna()
    #schedule = schedule.append(pd.Series('Ostiano' , [len(schedule)]))
    #print(schedule)
    #Otteniamo la schedulazione filtrando i nan e salviamo anche il suo tempo di percorrenza
    best_schedule = pd.Series(schedule.dropna())
    best_time = schedule_time(schedule, dfd)
    local_best_time = best_time
    #Per ogni i e j effettua il 2-opt e salva se il risultato migliora con lo scambio
    #Alla fine, avendo provato a scambiare tutte el coppie e avendole confrontate otteremo la miglior schedulazione
    #per ogni giro di 2-opt parte dalla schedulazione greedy
    while(True):
        for i in range(1, len(schedule)-2):
            for j in range(i+1, len(schedule)-1):
                s = two_opt(pd.Series(schedule), i, j, dfd)
                t = sum(s.values)
                if schedule_time(best_schedule, dfd)> t:
                    best_time = t.copy()
                    best_schedule = cities_list(s) 
        if(local_best_time>best_time):
            schedule = best_schedule[:]
            local_best_time = best_time
        else:
            break
    df = pd.concat([df, pd.DataFrame(pd.Series(best_schedule).rename(day))], axis=1)       
df.to_csv('../Dati/turni_f/optimized_start/oss_2opt_km_f1.csv', index=False, sep=";", encoding = "ISO-8859-1")

lunedi
martedi
mercoledi
giovedi
venerdi


In [12]:
week= ['martedi', 'mercoledi', 'giovedi']
df = pd.DataFrame({})
for day in week:
    print(day)
    #schedule = pd.Series('Cremona, sesto 39')
    schedule = fur2d[day].dropna()
    #schedule = schedule.append(pd.Series('San Giovanni in Croce, Feudatari, 25 A' , [len(schedule)]))
    #print(schedule)
    best_schedule = pd.Series(schedule.dropna())
    best_time = schedule_time(schedule, dfd)
    local_best_time = best_time
    while(True):
        for i in range(1, len(schedule)-2):
            for j in range(i+1, len(schedule)-1):
                s = two_opt(pd.Series(schedule), i, j, dfd)
                t = sum(s.values)
                if schedule_time(best_schedule, dfd)> t:
                    best_time = t.copy()
                    best_schedule = cities_list(s) 
        if(local_best_time>best_time):
            schedule = best_schedule[:]
            local_best_time = best_time
        else:
            break 
    df = pd.concat([df, pd.DataFrame(pd.Series(best_schedule).rename(day))], axis=1)        
df.to_csv('../Dati/turni_f/optimized_start/oss_2opt_km_f2.csv', index=False, sep=";", encoding = "ISO-8859-1")

martedi
mercoledi
giovedi
